# Scraping Data Using Foursquare API

### Import Necessary Library

In [1]:
# load necessary library
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim

### Get access to Foursquare API

In [ ]:
CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare Secret
ACCESS_TOKEN = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare access token
VERSION = '20210101' # Foursquare API version
LIMIT = 50 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Define Location

In [3]:
# I'm choosing Universitas Komputer Indonesia
address = 'Universitas Komputer Indonesia, Jawa Barat, Indonesia'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# print coordinate location
print(latitude, longitude)

-6.8868408 107.6151136


### Search Hotel around with radius 500 meters

In [5]:
# define parameters
search_key = 'hotel'
radius = 500

# define url
url_key = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude,
    ACCESS_TOKEN, VERSION, search_key, radius, LIMIT)

# get results based url_key
results = requests.get(url_key).json()

# assign relevant part of JSON to hotels
hotels = results['response']['venues']

# tranform venues into a dataframe
df_hotel = pd.json_normalize(hotels)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['id','name'] + [col for col in df_hotel.columns if col.startswith('location.')]
hotel_filtered = df_hotel.loc[:, filtered_columns]

# clean column names by keeping only last term
hotel_filtered.columns = [column.split('.')[-1] for column in hotel_filtered.columns]
df_hotel_filtered = hotel_filtered[['id', 'name', 'address', 'lat', 'lng']]

df_hotel_filtered.head()

,id,name,address,lat,lng
0,4c72303f9e706dcbf3374a62,Hotel Neo Dipatiukur,Jl. Dipatiukur No. 72-74,-6.889919,107.616451
1,4bb73d162f70c9b68e668630,Scarlet Dago Hotel,Jalan Siliwangi No. 5,-6.885105,107.612117
2,4be81e5dd837c9b6efe4a406,Royal Dago Hotel,Jl. Ir. H. Djuanda No. 156-169,-6.886584,107.613328
3,515e6be5e4b0c134437ee63a,Patra Jasa Hotel,Jl. Ir. H. Juanda No.132,-6.889154,107.614085
4,4dfef268483b8f033b425a03,Hotel Puspa Kencana,Jl. Siliwangi no. 10,-6.884576,107.612229


### Add places near the hotel

In [6]:
def get_nearby_hotels(latitudes, longitudes, radius=500):
    url_nearby = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit=30'.format(
        CLIENT_ID, CLIENT_SECRET, latitudes, longitudes, VERSION, radius)
    
    res_nearby = requests.get(url_nearby).json()["response"]['groups'][0]['items']

    # return only relevant information for each nearby venue
    df_nearby = pd.DataFrame([(v['venue']['name'], 
                               v['venue']['location']['lat'],
                               v['venue']['location']['lng'],
                               v['venue']['categories'][0]['name']) for v in res_nearby])
    df_nearby.columns = ['Venue Name', 'Venue Latitude', 'Venua Longitude', 'Venue Category']

    return df_nearby

In [7]:
list_nearby = []
for hotel, lat, lng, in zip(df_hotel_filtered.name, df_hotel_filtered.lat, df_hotel_filtered.lng):
    nearby_hotels = get_nearby_hotels(lat, lng)
    nearby_hotels['Hotel Name'] = hotel
    nearby_hotels['Hotel Latitude'] = lat
    nearby_hotels['Hotel Longitude'] = lng
    list_nearby.append(nearby_hotels)
       
df_nearby_hotels = pd.concat(list_nearby)
df_nearby_hotels = df_nearby_hotels[df_nearby_hotels['Venue Category'] != 'Hotel'].reset_index(drop=True)
df_nearby_hotels.head()

,Venue Name,Venue Latitude,Venua Longitude,Venue Category,Hotel Name,Hotel Latitude,Hotel Longitude
0,Koffie Opa,-6.891955,107.616803,Coffee Shop,Hotel Neo Dipatiukur,-6.889919,107.616451
1,Mie Merapi,-6.891459,107.616998,Noodle House,Hotel Neo Dipatiukur,-6.889919,107.616451
2,Bebeke Om Aris,-6.892662,107.617820,Asian Restaurant,Hotel Neo Dipatiukur,-6.889919,107.616451
3,Caffétto - Affection By Coffee,-6.888999,107.618090,Coffee Shop,Hotel Neo Dipatiukur,-6.889919,107.616451
4,DU 71a,-6.889382,107.615431,Café,Hotel Neo Dipatiukur,-6.889919,107.616451
